In [1]:
import pandas as pd
import numpy as np

In [2]:
advbs = pd.read_csv("AdvancedGameStats.csv")
print(advbs.shape)
advbs.head()

(16658, 87)


,Unnamed: 0,SEASON_YEAR,GAME_ID,GAME_DATE,MIN,H_TEAM_ID,H_TEAM_ABBREVIATION,H_TEAM_NAME,A_TEAM_ID,A_TEAM_ABBREVIATION,...,A_AST/TO,A_ASTRatio,A_OREB%,A_DREB%,A_REB%,A_TOV%,A_eFG%,A_TS%,A_PACE,A_PIE
0,0,2010-11,21000001,2010-10-26,48.0,1610612738,BOS,Boston Celtics,1610612748,MIA,...,0.88,12.7,25.0,73.2,47.2,18.7,41.9,47.1,91.5,40.1
1,1,2010-11,21000002,2010-10-26,48.0,1610612757,POR,Portland Trail Blazers,1610612756,PHX,...,0.79,13.2,24.4,51.0,39.1,21.1,54.7,56.8,90.0,37.5
2,2,2010-11,21000003,2010-10-26,48.0,1610612747,LAL,Los Angeles Lakers,1610612745,HOU,...,1.19,16.9,39.6,69.5,55.4,20.4,46.2,53.2,103.5,50.7
3,3,2010-11,21000004,2010-10-27,48.0,1610612739,CLE,Cleveland Cavaliers,1610612738,BOS,...,1.26,19.4,22.5,70.8,48.9,20.7,49.3,53.5,92.0,47.7
4,4,2010-11,21000005,2010-10-27,48.0,1610612751,NJN,New Jersey Nets,1610612765,DET,...,3.50,16.8,28.8,68.2,46.9,6.7,49.4,50.9,89.5,48.1


In [3]:
percent_cols = [col for col in advbs.columns if col.endswith('%')]

advbs[percent_cols] = advbs[percent_cols] / 100

In [4]:
advbs = advbs.drop(columns=['Unnamed: 0', 'AVAILABLE_FLAG'])

In [5]:
seasons = ['2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
advbs = advbs[(advbs['SEASON_YEAR'].isin(seasons))]
print(advbs.shape)

(5829, 85)


In [6]:
regbs = pd.read_csv("regular_season_totals_2010_2024.csv")
regbs = regbs.sort_values(by='GAME_ID')
regbs = regbs[(regbs['SEASON_YEAR'].isin(seasons))]
regbs = regbs.drop(columns=[col for col in regbs.columns if "RANK" in col])
regbs.head()

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,AVAILABLE_FLAG
1281,2019-20,1610612740,NOP,New Orleans Pelicans,21900001,2019-10-22T00:00:00,NOP @ TOR,L,53.0,43,...,30,19.0,4,9,3,34,24,122,-8.0,1.0
9450,2019-20,1610612761,TOR,Toronto Raptors,21900001,2019-10-22T00:00:00,TOR vs. NOP,W,53.0,42,...,23,17.0,7,3,9,24,34,130,8.0,1.0
12739,2019-20,1610612747,LAL,Los Angeles Lakers,21900002,2019-10-22T00:00:00,LAL @ LAC,L,48.0,37,...,20,15.0,4,7,5,24,25,102,-10.0,1.0
18446,2019-20,1610612746,LAC,LA Clippers,21900002,2019-10-22T00:00:00,LAC vs. LAL,W,48.0,42,...,24,14.0,8,5,7,25,24,112,10.0,1.0
123,2019-20,1610612766,CHA,Charlotte Hornets,21900003,2019-10-23T00:00:00,CHA vs. CHI,W,48.0,45,...,28,19.0,3,4,4,18,20,126,1.0,1.0


In [7]:
advboxscores = []

for season in regbs.SEASON_YEAR.unique():
    df = pd.read_csv(f"advboxscores/advboxscores{season}.csv")
    advboxscores.append(df)

advstats = pd.concat(advboxscores, ignore_index=True)

In [8]:
advstats.rename(columns={"Match\xa0Up": "MATCHUP", "Game\xa0Date": "GAME_DATE"}, inplace=True)
regbs['GAME_DATE'] = pd.to_datetime(regbs['GAME_DATE'])
advstats['GAME_DATE'] = pd.to_datetime(advstats['GAME_DATE'])

In [9]:
rolling = pd.merge(
    regbs,
    advstats.drop(columns=['MIN', 'W/L']),
    on=['MATCHUP', 'GAME_DATE'],
    how='left'
)

In [10]:
rolling = rolling.drop(columns=['AVAILABLE_FLAG'])
rolling.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11658 entries, 0 to 11657
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   SEASON_YEAR        11658 non-null  object        
 1   TEAM_ID            11658 non-null  int64         
 2   TEAM_ABBREVIATION  11658 non-null  object        
 3   TEAM_NAME          11658 non-null  object        
 4   GAME_ID            11658 non-null  int64         
 5   GAME_DATE          11658 non-null  datetime64[ns]
 6   MATCHUP            11658 non-null  object        
 7   WL                 11658 non-null  object        
 8   MIN                11658 non-null  float64       
 9   FGM                11658 non-null  int64         
 10  FGA                11658 non-null  int64         
 11  FG_PCT             11658 non-null  float64       
 12  FG3M               11658 non-null  int64         
 13  FG3A               11658 non-null  int64         
 14  FG3_PC

In [11]:
percent_cols = [col for col in rolling.columns if col.endswith('%')]

rolling[percent_cols] = rolling[percent_cols] / 100

In [12]:
rolling.to_csv('Rolling.csv')

In [13]:
def rolling_average(df, metric, window=5):
    rolling_series = (
        df.groupby('TEAM_ID')[metric]
        .rolling(window, min_periods=window)  
        .mean()
        .shift(1)
        .reset_index(level=0, drop=True)
    )
    rolling_series.loc[df.groupby('TEAM_ID').head(4).index] = np.nan
    
    return rolling_series

In [14]:
rolling_numeric = rolling.select_dtypes(include=['int', 'float']).drop(columns=['TEAM_ID', 'GAME_ID', 'MIN'])

averages_5 = rolling_numeric.apply(lambda col: rolling_average(rolling, col.name, window=5))
averages_5.columns = [f"{col}_AVG_5" for col in averages_5.columns]

merge_info = ['GAME_DATE', 'TEAM_ID']
test = pd.concat([rolling[merge_info], averages_5], axis=1)

In [15]:
averages_10 = rolling_numeric.apply(lambda col: rolling_average(rolling, col.name, window=10))
averages_10.columns = [f"{col}_AVG_10" for col in averages_10.columns]

test = pd.concat([test, averages_10], axis=1)

In [16]:
advbs['GAME_DATE'] = pd.to_datetime(advbs['GAME_DATE'])

In [17]:
advbs = advbs.merge(
    test,
    left_on=['GAME_DATE', 'H_TEAM_ID'],
    right_on=['GAME_DATE', 'TEAM_ID'],
    how='left',
    suffixes=('', '_H')
)

advbs = advbs.merge(
    test,
    left_on=['GAME_DATE', 'A_TEAM_ID'],
    right_on=['GAME_DATE', 'TEAM_ID'],
    how='left',
    suffixes=('', '_A')
)

for col in advbs.columns:
    if col.endswith('_5') or col.endswith('_10'):
        advbs.rename(columns={col: 'H_' + col}, inplace=True)
    elif col.endswith('_A'):
        sub = len('_A')
        advbs.rename(columns={col: 'A_' + col[:-sub]}, inplace=True)

In [18]:
advbs = advbs.drop(columns=['TEAM_ID'])
advbs.columns.tolist()

['SEASON_YEAR',
 'GAME_ID',
 'GAME_DATE',
 'MIN',
 'H_TEAM_ID',
 'H_TEAM_ABBREVIATION',
 'H_TEAM_NAME',
 'A_TEAM_ID',
 'A_TEAM_ABBREVIATION',
 'A_TEAM_NAME',
 'MATCHUP',
 'H_WL',
 'H_FGM',
 'H_FGA',
 'H_FG_PCT',
 'H_FG3M',
 'H_FG3A',
 'H_FG3_PCT',
 'H_FTM',
 'H_FTA',
 'H_FT_PCT',
 'H_OREB',
 'H_DREB',
 'H_REB',
 'H_AST',
 'H_TOV',
 'H_STL',
 'H_BLK',
 'H_BLKA',
 'H_PF',
 'H_PFD',
 'H_PTS',
 'H_PLUS_MINUS',
 'A_FGM',
 'A_FGA',
 'A_FG_PCT',
 'A_FG3M',
 'A_FG3A',
 'A_FG3_PCT',
 'A_FTM',
 'A_FTA',
 'A_FT_PCT',
 'A_OREB',
 'A_DREB',
 'A_REB',
 'A_AST',
 'A_TOV',
 'A_STL',
 'A_BLK',
 'A_BLKA',
 'A_PF',
 'A_PFD',
 'A_PTS',
 'A_PLUS_MINUS',
 'SPREAD',
 'TOTAL',
 'OREB',
 'H_OffRtg',
 'H_DefRtg',
 'H_NetRtg',
 'H_AST%',
 'H_AST/TO',
 'H_ASTRatio',
 'H_OREB%',
 'H_DREB%',
 'H_REB%',
 'H_TOV%',
 'H_eFG%',
 'H_TS%',
 'H_PACE',
 'H_PIE',
 'A_OffRtg',
 'A_DefRtg',
 'A_NetRtg',
 'A_AST%',
 'A_AST/TO',
 'A_ASTRatio',
 'A_OREB%',
 'A_DREB%',
 'A_REB%',
 'A_TOV%',
 'A_eFG%',
 'A_TS%',
 'A_PACE',
 'A_PIE

In [19]:
tor = (advbs.H_TEAM_ABBREVIATION == 'TOR') | (advbs.A_TEAM_ABBREVIATION == 'TOR')

In [20]:
pd.set_option('display.max_columns', None)
advbs[tor].head(11)

,SEASON_YEAR,GAME_ID,GAME_DATE,MIN,H_TEAM_ID,H_TEAM_ABBREVIATION,H_TEAM_NAME,A_TEAM_ID,A_TEAM_ABBREVIATION,A_TEAM_NAME,MATCHUP,H_WL,H_FGM,H_FGA,H_FG_PCT,H_FG3M,H_FG3A,H_FG3_PCT,H_FTM,H_FTA,H_FT_PCT,H_OREB,H_DREB,H_REB,H_AST,H_TOV,H_STL,H_BLK,H_BLKA,H_PF,H_PFD,H_PTS,H_PLUS_MINUS,A_FGM,A_FGA,A_FG_PCT,A_FG3M,A_FG3A,A_FG3_PCT,A_FTM,A_FTA,A_FT_PCT,A_OREB,A_DREB,A_REB,A_AST,A_TOV,A_STL,A_BLK,A_BLKA,A_PF,A_PFD,A_PTS,A_PLUS_MINUS,SPREAD,TOTAL,OREB,H_OffRtg,H_DefRtg,H_NetRtg,H_AST%,H_AST/TO,H_ASTRatio,H_OREB%,H_DREB%,H_REB%,H_TOV%,H_eFG%,H_TS%,H_PACE,H_PIE,A_OffRtg,A_DefRtg,A_NetRtg,A_AST%,A_AST/TO,A_ASTRatio,A_OREB%,A_DREB%,A_REB%,A_TOV%,A_eFG%,A_TS%,A_PACE,A_PIE,H_FGM_AVG_5,H_FGA_AVG_5,H_FG_PCT_AVG_5,H_FG3M_AVG_5,H_FG3A_AVG_5,H_FG3_PCT_AVG_5,H_FTM_AVG_5,H_FTA_AVG_5,H_FT_PCT_AVG_5,H_OREB_AVG_5,H_DREB_AVG_5,H_REB_AVG_5,H_AST_AVG_5,H_TOV_AVG_5,H_STL_AVG_5,H_BLK_AVG_5,H_BLKA_AVG_5,H_PF_AVG_5,H_PFD_AVG_5,H_PTS_AVG_5,H_PLUS_MINUS_AVG_5,H_OffRtg_AVG_5,H_DefRtg_AVG_5,H_NetRtg_AVG_5,H_AST%_AVG_5,H_AST/TO_AVG_5,H_ASTRatio_AVG_5,H_OREB%_AVG_5,H_DREB%_AVG_5,H_REB%_AVG_5,H_TOV%_AVG_5,H_eFG%_AVG_5,H_TS%_AVG_5,H_PACE_AVG_5,H_PIE_AVG_5,H_FGM_AVG_10,H_FGA_AVG_10,H_FG_PCT_AVG_10,H_FG3M_AVG_10,H_FG3A_AVG_10,H_FG3_PCT_AVG_10,H_FTM_AVG_10,H_FTA_AVG_10,H_FT_PCT_AVG_10,H_OREB_AVG_10,H_DREB_AVG_10,H_REB_AVG_10,H_AST_AVG_10,H_TOV_AVG_10,H_STL_AVG_10,H_BLK_AVG_10,H_BLKA_AVG_10,H_PF_AVG_10,H_PFD_AVG_10,H_PTS_AVG_10,H_PLUS_MINUS_AVG_10,H_OffRtg_AVG_10,H_DefRtg_AVG_10,H_NetRtg_AVG_10,H_AST%_AVG_10,H_AST/TO_AVG_10,H_ASTRatio_AVG_10,H_OREB%_AVG_10,H_DREB%_AVG_10,H_REB%_AVG_10,H_TOV%_AVG_10,H_eFG%_AVG_10,H_TS%_AVG_10,H_PACE_AVG_10,H_PIE_AVG_10,A_TEAM_ID,A_FGM_AVG_5,A_FGA_AVG_5,A_FG_PCT_AVG_5,A_FG3M_AVG_5,A_FG3A_AVG_5,A_FG3_PCT_AVG_5,A_FTM_AVG_5,A_FTA_AVG_5,A_FT_PCT_AVG_5,A_OREB_AVG_5,A_DREB_AVG_5,A_REB_AVG_5,A_AST_AVG_5,A_TOV_AVG_5,A_STL_AVG_5,A_BLK_AVG_5,A_BLKA_AVG_5,A_PF_AVG_5,A_PFD_AVG_5,A_PTS_AVG_5,A_PLUS_MINUS_AVG_5,A_OffRtg_AVG_5,A_DefRtg_AVG_5,A_NetRtg_AVG_5,A_AST%_AVG_5,A_AST/TO_AVG_5,A_ASTRatio_AVG_5,A_OREB%_AVG_5,A_DREB%_AVG_5,A_REB%_AVG_5,A_TOV%_AVG_5,A_eFG%_AVG_5,A_TS%_AVG_5,A_PACE_AVG_5,A_PIE_AVG_5,A_FGM_AVG_10,A_FGA_AVG_10,A_FG_PCT_AVG_10,A_FG3M_AVG_10,A_FG3A_AVG_10,A_FG3_PCT_AVG_10,A_FTM_AVG_10,A_FTA_AVG_10,A_FT_PCT_AVG_10,A_OREB_AVG_10,A_DREB_AVG_10,A_REB_AVG_10,A_AST_AVG_10,A_TOV_AVG_10,A_STL_AVG_10,A_BLK_AVG_10,A_BLKA_AVG_10,A_PF_AVG_10,A_PFD_AVG_10,A_PTS_AVG_10,A_PLUS_MINUS_AVG_10,A_OffRtg_AVG_10,A_DefRtg_AVG_10,A_NetRtg_AVG_10,A_AST%_AVG_10,A_AST/TO_AVG_10,A_ASTRatio_AVG_10,A_OREB%_AVG_10,A_DREB%_AVG_10,A_REB%_AVG_10,A_TOV%_AVG_10,A_eFG%_AVG_10,A_TS%_AVG_10,A_PACE_AVG_10,A_PIE_AVG_10
0,2019-20,21900001,2019-10-22,53.0,1610612761,TOR,Toronto Raptors,1610612740,NOP,New Orleans Pelicans,TOR vs. NOP,W,42,103,0.408,14,40,0.350,32,38,0.842,16,41,57,23,17.0,7,3,9,24,34,130,8.0,43,102,0.422,19,45,0.422,17,20,0.850,16,37,53,30,19.0,4,9,3,34,24,122,-8.0,8,252,32,113.0,108.9,4.1,0.548,1.35,14.2,0.400,0.729,0.556,0.148,0.476,0.543,102.79,53.1,108.9,113.0,-4.1,0.698,1.58,19.0,0.271,0.600,0.444,0.170,0.515,0.551,102.79,46.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1610612740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,2019-20,21900017,2019-10-25,48.0,1610612738,BOS,Boston Celtics,1610612761,TOR,Toronto Raptors,BOS vs. TOR,W,42,109,0.385,13,38,0.342,15,16,0.938,21,35,56,21,10.0,10,8,6,20,28,112,6.0,37,78,0.474,18,36,0.500,14,17,0.824,5,41,46,22,23.0,2,6,8,28,20,106,-6.0,6,218,26,106.7,101.9,4.7,0.500,2.10,14.0,0.353,0.860,0.550,0.095,0.445,0.483,104.50,53.7,101.9,106.7,-4.7,0.595,0.96,16.7,0

In [21]:
def momentum_indicator(df, metric):
    """Compute momentum as the difference between 5-game and 10-game rolling averages."""
    return (df[metric] - df[metric.replace("AVG_5", "AVG_10")]) / df[metric.replace("AVG_5", "AVG_10")]

In [22]:
advbs_rolling = advbs[[col for col in advbs.columns if col.endswith("_AVG_5") or col.endswith("_AVG_10")]]

indicators = advbs_rolling.apply(lambda col: momentum_indicator(advbs_rolling, col.name) if col.name.endswith("_AVG_5") else None)
indicators = indicators.dropna(axis=1, how='all')

In [23]:
indicators.columns = [col.replace('AVG_5', 'MOMENTUM') for col in indicators.columns]

In [24]:
indicators = pd.concat([advbs['GAME_ID'], indicators], axis=1)

In [25]:
advbs = advbs.merge(
    indicators,
    on='GAME_ID',
    how='left'
)

In [26]:
advbs = advbs.loc[:, ~advbs.columns.duplicated()]

In [27]:
rolling.WL = rolling.WL.map({'W': 1, 'L': 0})

In [28]:
def win_pct(df, team, season):
    mask = (df.TEAM_ABBREVIATION == team) & (df.SEASON_YEAR == season)
    reduced = df[mask].sort_values(by='GAME_DATE')
    return reduced.WL.cumsum().shift(1) / reduced.WL.expanding().count().shift(1)

In [29]:
rolling['WIN_PCT_PRE'] = (
    rolling.groupby(['TEAM_ABBREVIATION', 'SEASON_YEAR'])
    .apply(lambda x: win_pct(x, x.TEAM_ABBREVIATION.iloc[0], x.SEASON_YEAR.iloc[0]))
    .reset_index(level=[0,1], drop=True)
)

In [30]:
win_pct_df = rolling[['GAME_DATE', 'TEAM_ID', 'WIN_PCT_PRE']]

advbs = advbs.merge(
    win_pct_df, 
    left_on=['GAME_DATE', 'H_TEAM_ID'], 
    right_on=['GAME_DATE', 'TEAM_ID'], 
    how='left'
).rename(columns={'WIN_PCT_PRE': 'H_WIN_PCT'}).drop(columns=['TEAM_ID'])

advbs = advbs.merge(
    win_pct_df, 
    left_on=['GAME_DATE', 'A_TEAM_ID'], 
    right_on=['GAME_DATE', 'TEAM_ID'], 
    how='left'
).rename(columns={'WIN_PCT_PRE': 'A_WIN_PCT'}).drop(columns=['TEAM_ID'])

In [31]:
advbs = advbs.sort_values(by='GAME_DATE')

In [32]:
pd.set_option('display.max_columns', None)
bos = (advbs.H_TEAM_ABBREVIATION == 'BOS') | (advbs.A_TEAM_ABBREVIATION == 'BOS')
advbs[bos].head(11)

,SEASON_YEAR,GAME_ID,GAME_DATE,MIN,H_TEAM_ID,H_TEAM_ABBREVIATION,H_TEAM_NAME,A_TEAM_ID,A_TEAM_ABBREVIATION,A_TEAM_NAME,MATCHUP,H_WL,H_FGM,H_FGA,H_FG_PCT,H_FG3M,H_FG3A,H_FG3_PCT,H_FTM,H_FTA,H_FT_PCT,H_OREB,H_DREB,H_REB,H_AST,H_TOV,H_STL,H_BLK,H_BLKA,H_PF,H_PFD,H_PTS,H_PLUS_MINUS,A_FGM,A_FGA,A_FG_PCT,A_FG3M,A_FG3A,A_FG3_PCT,A_FTM,A_FTA,A_FT_PCT,A_OREB,A_DREB,A_REB,A_AST,A_TOV,A_STL,A_BLK,A_BLKA,A_PF,A_PFD,A_PTS,A_PLUS_MINUS,SPREAD,TOTAL,OREB,H_OffRtg,H_DefRtg,H_NetRtg,H_AST%,H_AST/TO,H_ASTRatio,H_OREB%,H_DREB%,H_REB%,H_TOV%,H_eFG%,H_TS%,H_PACE,H_PIE,A_OffRtg,A_DefRtg,A_NetRtg,A_AST%,A_AST/TO,A_ASTRatio,A_OREB%,A_DREB%,A_REB%,A_TOV%,A_eFG%,A_TS%,A_PACE,A_PIE,H_FGM_AVG_5,H_FGA_AVG_5,H_FG_PCT_AVG_5,H_FG3M_AVG_5,H_FG3A_AVG_5,H_FG3_PCT_AVG_5,H_FTM_AVG_5,H_FTA_AVG_5,H_FT_PCT_AVG_5,H_OREB_AVG_5,H_DREB_AVG_5,H_REB_AVG_5,H_AST_AVG_5,H_TOV_AVG_5,H_STL_AVG_5,H_BLK_AVG_5,H_BLKA_AVG_5,H_PF_AVG_5,H_PFD_AVG_5,H_PTS_AVG_5,H_PLUS_MINUS_AVG_5,H_OffRtg_AVG_5,H_DefRtg_AVG_5,H_NetRtg_AVG_5,H_AST%_AVG_5,H_AST/TO_AVG_5,H_ASTRatio_AVG_5,H_OREB%_AVG_5,H_DREB%_AVG_5,H_REB%_AVG_5,H_TOV%_AVG_5,H_eFG%_AVG_5,H_TS%_AVG_5,H_PACE_AVG_5,H_PIE_AVG_5,H_FGM_AVG_10,H_FGA_AVG_10,H_FG_PCT_AVG_10,H_FG3M_AVG_10,H_FG3A_AVG_10,H_FG3_PCT_AVG_10,H_FTM_AVG_10,H_FTA_AVG_10,H_FT_PCT_AVG_10,H_OREB_AVG_10,H_DREB_AVG_10,H_REB_AVG_10,H_AST_AVG_10,H_TOV_AVG_10,H_STL_AVG_10,H_BLK_AVG_10,H_BLKA_AVG_10,H_PF_AVG_10,H_PFD_AVG_10,H_PTS_AVG_10,H_PLUS_MINUS_AVG_10,H_OffRtg_AVG_10,H_DefRtg_AVG_10,H_NetRtg_AVG_10,H_AST%_AVG_10,H_AST/TO_AVG_10,H_ASTRatio_AVG_10,H_OREB%_AVG_10,H_DREB%_AVG_10,H_REB%_AVG_10,H_TOV%_AVG_10,H_eFG%_AVG_10,H_TS%_AVG_10,H_PACE_AVG_10,H_PIE_AVG_10,A_FGM_AVG_5,A_FGA_AVG_5,A_FG_PCT_AVG_5,A_FG3M_AVG_5,A_FG3A_AVG_5,A_FG3_PCT_AVG_5,A_FTM_AVG_5,A_FTA_AVG_5,A_FT_PCT_AVG_5,A_OREB_AVG_5,A_DREB_AVG_5,A_REB_AVG_5,A_AST_AVG_5,A_TOV_AVG_5,A_STL_AVG_5,A_BLK_AVG_5,A_BLKA_AVG_5,A_PF_AVG_5,A_PFD_AVG_5,A_PTS_AVG_5,A_PLUS_MINUS_AVG_5,A_OffRtg_AVG_5,A_DefRtg_AVG_5,A_NetRtg_AVG_5,A_AST%_AVG_5,A_AST/TO_AVG_5,A_ASTRatio_AVG_5,A_OREB%_AVG_5,A_DREB%_AVG_5,A_REB%_AVG_5,A_TOV%_AVG_5,A_eFG%_AVG_5,A_TS%_AVG_5,A_PACE_AVG_5,A_PIE_AVG_5,A_FGM_AVG_10,A_FGA_AVG_10,A_FG_PCT_AVG_10,A_FG3M_AVG_10,A_FG3A_AVG_10,A_FG3_PCT_AVG_10,A_FTM_AVG_10,A_FTA_AVG_10,A_FT_PCT_AVG_10,A_OREB_AVG_10,A_DREB_AVG_10,A_REB_AVG_10,A_AST_AVG_10,A_TOV_AVG_10,A_STL_AVG_10,A_BLK_AVG_10,A_BLKA_AVG_10,A_PF_AVG_10,A_PFD_AVG_10,A_PTS_AVG_10,A_PLUS_MINUS_AVG_10,A_OffRtg_AVG_10,A_DefRtg_AVG_10,A_NetRtg_AVG_10,A_AST%_AVG_10,A_AST/TO_AVG_10,A_ASTRatio_AVG_10,A_OREB%_AVG_10,A_DREB%_AVG_10,A_REB%_AVG_10,A_TOV%_AVG_10,A_eFG%_AVG_10,A_TS%_AVG_10,A_PACE_AVG_10,A_PIE_AVG_10,H_FGM_MOMENTUM,H_FGA_MOMENTUM,H_FG_PCT_MOMENTUM,H_FG3M_MOMENTUM,H_FG3A_MOMENTUM,H_FG3_PCT_MOMENTUM,H_FTM_MOMENTUM,H_FTA_MOMENTUM,H_FT_PCT_MOMENTUM,H_OREB_MOMENTUM,H_DREB_MOMENTUM,H_REB_MOMENTUM,H_AST_MOMENTUM,H_TOV_MOMENTUM,H_STL_MOMENTUM,H_BLK_MOMENTUM,H_BLKA_MOMENTUM,H_PF_MOMENTUM,H_PFD_MOMENTUM,H_PTS_MOMENTUM,H_PLUS_MINUS_MOMENTUM,H_OffRtg_MOMENTUM,H_DefRtg_MOMENTUM,H_NetRtg_MOMENTUM,H_AST%_MOMENTUM,H_AST/TO_MOMENTUM,H_ASTRatio_MOMENTUM,H_OREB%_MOMENTUM,H_DREB%_MOMENTUM,H_REB%_MOMENTUM,H_TOV%_MOMENTUM,H_eFG%_MOMENTUM,H_TS%_MOMENTUM,H_PACE_MOMENTUM,H_PIE_MOMENTUM,A_FGM_MOMENTUM,A_FGA_MOMENTUM,A_FG_PCT_MOMENTUM,A_FG3M_MOMENTUM,A_FG3A_MOMENTUM,A_FG3_PCT_MOMENTUM,A_FTM_MOMENTUM,A_FTA_MOMENTUM,A_FT_PCT_MOMENTUM,A_OREB_MOMENTUM,A_DREB_MOMENTUM,A_REB_MOMENTUM,A_AST_MOMENTUM,A_TOV_MOMENTUM,A_STL_MOMENTUM,A_BLK_MOMENTUM,A_BLKA_MOMENTUM,A_PF_MOMENTUM,A_PFD_MOMENTUM,A_PTS_MOMENTUM,A_PLUS_MINUS_MOMENTUM,A_OffRtg_MOMENTUM,A_DefRtg_MOMENTUM,A_NetRtg_MOMENTUM,A_AST%_MOMENTUM,A_AST/TO_MOMENTUM,A_ASTRatio_MOMENTUM,A_OREB%_MOMENTUM,A_DREB%_MOMENTUM,A_REB%_MOMENTUM,A_TOV%_MOMENTUM,A_eFG%_MOMENTUM,A_TS%_MOMENTUM,A_PACE_MOMENTUM,A_PIE_MOMENTUM,H_WIN_PCT,A_WIN_PCT
7,2019-20,21900008,2019-10-23,48.0,1610612755,PHI,Philadelphia 76ers,1610612738,BOS,Boston Celtics,PHI vs. BOS,W,37,85,0.435,7,29,0.241,26,36,0.722,12,50,62,24,17.0,6,7,2,34,29,107,14.0,33,90,0.367,7,26,0.269,20,34,0.588,10,31,41,18,1

In [33]:
pd.reset_option('all')

In [34]:
advbs.to_csv('ARMGameStats.csv')